In [1]:
import pandas as pd

data = pd.read_csv('/Users/tyco/Desktop/MADS/sanovo/data/raw/HSTMachineError.csv')
data.head(5)

,MerId,MerType,MerDate,MerNumber,MerSubNumber,MerData1,MerData2,MerData3,MerData4,MerData5,MerMinId,MerText,MerDeviceType,MerDeviceNumber,MerPriority
0,11897185,7,2025-01-30 10:22:00.000,10,22,181.0,180.0,90.0,NaN,NaN,4620,NaN,0,0,0
1,11897186,21,2025-01-30 10:21:42.000,460,12,1.0,22.0,41.0,NaN,NaN,4620,NaN,0,0,0
2,11897187,23,2025-01-30 10:21:56.000,460,12,0.0,0.0,0.0,NaN,NaN,4620,NaN,0,0,0
3,11897188,21,2025-01-30 10:21:56.000,460,12,1.0,22.0,41.0,NaN,NaN,4620,NaN,0,0,0
4,11897189,23,2025-01-30 10:21:57.000,460,12,0.0,0.0,0.0,NaN,NaN,4620,NaN,0,0,0


In [2]:
# Create a copy of the data
data_copy = data.copy()

We filter the data for lane errors/warnings and machine stops

In [7]:
data_filtered = data_copy[(data_copy['MerType'].between(21, 24)) | (data_copy['MerType'] == 8)]
data_filtered.head(5)

,MerId,MerType,MerDate,MerNumber,MerSubNumber,MerData1,MerData2,MerData3,MerData4,MerData5,MerMinId,MerText,MerDeviceType,MerDeviceNumber,MerPriority
1,11897186,21,2025-01-30 10:21:42.000,460,12,1.0,22.0,41.0,NaN,NaN,4620,NaN,0,0,0
2,11897187,23,2025-01-30 10:21:56.000,460,12,0.0,0.0,0.0,NaN,NaN,4620,NaN,0,0,0
3,11897188,21,2025-01-30 10:21:56.000,460,12,1.0,22.0,41.0,NaN,NaN,4620,NaN,0,0,0
4,11897189,23,2025-01-30 10:21:57.000,460,12,0.0,0.0,0.0,NaN,NaN,4620,NaN,0,0,0
5,11897190,21,2025-01-30 10:21:57.000,460,12,1.0,22.0,41.0,NaN,NaN,4620,NaN,0,0,0


We filter columns that are not relevant and set the MerDate as index

In [8]:
data_filtered = data_filtered.drop(columns=['MerId', 'MerData4', 'MerData5', 'MerMinId', 'MerText', 'MerDeviceType', 'MerDeviceNumber', 'MerPriority'])
data_filtered = data_filtered.set_index('MerDate')
data_filtered.head(5)

,MerType,MerNumber,MerSubNumber,MerData1,MerData2,MerData3
MerDate,,,,,,
2025-01-30 10:21:42.000,21,460,12,1.0,22.0,41.0
2025-01-30 10:21:56.000,23,460,12,0.0,0.0,0.0
2025-01-30 10:21:56.000,21,460,12,1.0,22.0,41.0
2025-01-30 10:21:57.000,23,460,12,0.0,0.0,0.0
2025-01-30 10:21:57.000,21,460,12,1.0,22.0,41.0


In [26]:
# Add a new column 'status' based on the conditions for 'MerType'
data_filtered['status'] = data_filtered['MerType'].map({
    21: 'error',
    22: 'warning',
    23: 'resolved',
    24: 'all errors/warnings lowered',
    8: 'machine stop'
})

data_filtered.head(5)

,MerType,MerNumber,MerSubNumber,MerData1,MerData2,MerData3,status,error_message,lane
MerDate,,,,,,,,,
2025-01-30 10:21:42.000,21,460,12,1.0,22.0,41.0,error,denester error,12
2025-01-30 10:21:56.000,23,460,12,0.0,0.0,0.0,resolved,denester error,12
2025-01-30 10:21:56.000,21,460,12,1.0,22.0,41.0,error,denester error,12
2025-01-30 10:21:57.000,23,460,12,0.0,0.0,0.0,resolved,denester error,12
2025-01-30 10:21:57.000,21,460,12,1.0,22.0,41.0,error,denester error,12


In [27]:
# Add a new column 'error_message' based on the conditions for 'MerNumber'
error_messages = {
    460: 'denester error',
    461: 'denester misplaced pack warning',
    452: 'packing lane double pack error',
    98: 'rerouting activated for eject station',
    414: 'cascade front door opened',
    411: 'general safety failure error',
    456: 'pack detection at setter error',
    490: 'pack final processing warning',
    453: 'missing eggs warning',
    96: 'pressure control activated for eject station',
    0: 'unknown error',
    1: 'watchdog error',
    458: 'missing egg detected',
    470: 'closer error',
    462: 'denester pack level low error',
    412: 'closer cover opened error',
    480: 'labeling error',
    483: 'labeling communication error',
    68: 'egg reservoir full at dropgate' ,
    415: 'denester 1 door opened error',
    420: 'release unit error',
    33: 'alarm begin-check max limit passed',
    401: 'unknown error',
    146: 'unknown error',
    191: 'unknown error',
    454: 'unknown error',
    92: 'unknown error',
    72: 'unknown error',
    430: 'unknown error',
    37: 'unknown error',
    12: 'unknown error',
    99: 'unknown error',
    413: 'unknown error',
    35: 'unknown error',
    221: 'unknown error',
    69: 'unknown error',
    173: 'unknown error',
    220: 'unknown error',
    120: 'unknown error',
    115: 'unknown error',
    109: 'unknown error',
    100: 'unknown error',
    196: 'unknown error',
    71: 'unknown error',
    108: 'unknown error',
    106: 'unknown error',
    107: 'unknown error',
    451: 'unknown error',
    481: 'unknown error',
    450: 'unknown error',
    421: 'unknown error',
    410: 'unknown error',
    124: 'unknown error',
    23: 'unknown error',
    197: 'unknown error',
    41: 'unknown error',
    40: 'unknown error',
    123: 'unknown error',
    57: 'unknown error',
    90: 'unknown error',
    114: 'unknown error',
    405: 'unknown error',
    13: 'unknown error',
    5: 'unknown error',
    455: 'unknown error',
    143: 'unknown error',
    147: 'unknown error',
    110: 'unknown error',
    181: 'unknown error',
    184: 'unknown error',
    201: 'unknown error',
    440: 'unknown error',
    162: 'unknown error',
    111: 'unknown error',
    85: 'unknown error',
    261: 'unknown error',
    511: 'unknown error',
    34: 'unknown error',
    144: 'unknown error',
    194: 'unknown error',
    163: 'unknown error',
    101: 'unknown error',
    83: 'unknown error',
    117: 'unknown error',
    87: 'unknown error',
    14: 'unknown error',
    160: 'unknown error',
    192: 'unknown error',
    21: 'unknown error',
    119: 'unknown error',
    77: 'unknown error',
    105: 'unknown error',
    172: 'unknown error',
    200: 'unknown error',
    127: 'unknown error',
    142: 'unknown error',
    102: 'unknown error',
    185: 'unknown error',
    140: 'unknown error',
    198: 'unknown error',
    104: 'unknown error',
    182: 'unknown error',
    183: 'unknown error'
}

data_filtered['error_message'] = data_filtered['MerNumber'].map(error_messages)
data_filtered.head(5)

,MerType,MerNumber,MerSubNumber,MerData1,MerData2,MerData3,status,error_message,lane
MerDate,,,,,,,,,
2025-01-30 10:21:42.000,21,460,12,1.0,22.0,41.0,error,denester error,12
2025-01-30 10:21:56.000,23,460,12,0.0,0.0,0.0,resolved,denester error,12
2025-01-30 10:21:56.000,21,460,12,1.0,22.0,41.0,error,denester error,12
2025-01-30 10:21:57.000,23,460,12,0.0,0.0,0.0,resolved,denester error,12
2025-01-30 10:21:57.000,21,460,12,1.0,22.0,41.0,error,denester error,12


In [23]:
# Set pandas display option to show 1000 rows
pd.set_option('display.max_rows', 10000)

In [30]:
# Add a new column 'lane' based on the condition for 'MerNumber'
data_filtered['lane'] = data_filtered.apply(lambda row: row['MerSubNumber'] if row['MerNumber'] >= 400 else 'machine error', axis=1)
data_filtered.head(5)

,MerType,MerNumber,MerSubNumber,MerData1,MerData2,MerData3,status,error_message,lane
MerDate,,,,,,,,,
2025-01-30 10:21:42.000,21,460,12,1.0,22.0,41.0,error,denester error,12
2025-01-30 10:21:56.000,23,460,12,0.0,0.0,0.0,resolved,denester error,12
2025-01-30 10:21:56.000,21,460,12,1.0,22.0,41.0,error,denester error,12
2025-01-30 10:21:57.000,23,460,12,0.0,0.0,0.0,resolved,denester error,12
2025-01-30 10:21:57.000,21,460,12,1.0,22.0,41.0,error,denester error,12


In [ ]:
data_filtered = data_filtered.drop(columns=['MerType', 'MerNumber', 'MerSubNumber', 'MerData1', 'MerData2', 'MerData3'])

In [33]:
data_filtered.head(2000)

,status,error_message,lane
MerDate,,,
2025-01-30 10:21:42.000,error,denester error,12
2025-01-30 10:21:56.000,resolved,denester error,12
2025-01-30 10:21:56.000,error,denester error,12
2025-01-30 10:21:57.000,resolved,denester error,12
2025-01-30 10:21:57.000,error,denester error,12
2025-01-30 10:21:58.000,resolved,denester error,12
2025-01-30 10:22:01.000,warning,denester misplaced pack warning,12
2025-01-30 10:22:03.000,error,denester error,12
2025-01-30 10:22:09.000,error,packing lane double pack error,12
